In [1]:
!conda install pandas -y

import pandas as pd

Solving environment: done

## Package Plan ##

  environment location: /Users/danielzafman/opt/miniconda3

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.12.5          |   py38hecd8cb5_0         141 KB
    conda-4.9.2                |   py38hecd8cb5_0         2.9 MB
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following packages will be UPDATED:

  certifi            pkgs/main/noarch::certifi-2020.6.20-p~ --> pkgs/main/osx-64::certifi-2020.12.5-py38hecd8cb5_0
  conda                                4.9.1-py38hecd8cb5_0 --> 4.9.2-py38hecd8cb5_0



conda-4.9.2          | 2.9 MB    | ##################################### | 100% 
certifi-2020.12.5    | 141 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying tran

In [12]:
team_stats_init = pd.read_csv("CSVS/Falcons.csv")

team_stats_init

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Offense.3,Offense.4,Defense,Defense.1,Defense.2,Defense.3,Defense.4,Expected Points,Expected Points.1,Expected Points.2
0,Week,Day,Date,NaN,NaN,NaN,OT,Rec,NaN,Opp,...,RushY,TO,1stD,TotYd,PassY,RushY,TO,Offense,Defense,Sp. Tms
1,1,Sun,September 8,1:00PM ET,boxscore,L,NaN,0-1,@,Minnesota Vikings,...,73,3,18,269,97,172,NaN,-1.83,-11.05,-4.34
2,2,Sun,September 15,8:20PM ET,boxscore,W,NaN,1-1,NaN,Philadelphia Eagles,...,57,3,18,286,237,49,3,3.96,5.22,-4.03
3,3,Sun,September 22,1:00PM ET,boxscore,L,NaN,1-2,@,Indianapolis Colts,...,93,1,29,379,300,79,NaN,20.23,-23.26,-2.98
4,4,Sun,September 29,1:00PM ET,boxscore,L,NaN,1-3,NaN,Tennessee Titans,...,58,1,22,365,227,138,NaN,1.83,-10.78,-3.80
5,5,Sun,October 6,1:00PM ET,boxscore,L,NaN,1-4,@,Houston Texans,...,57,1,31,592,426,166,1,6.40,-40.69,9.43
6,6,Sun,October 13,4:05PM ET,boxscore,L,NaN,1-5,@,Arizona Cardinals,...,103,NaN,24,442,340,102,NaN,24.99,-23.23,-0.25
7,7,Sun,October 20,1:00PM ET,boxscore,L,NaN,1-6,NaN,Los Angeles Rams,...,38,3,23,381,291,90,NaN,-10.97,-8.61,-7.98
8,8,Sun,October 27,1:00PM ET,boxscore,L,NaN,1-7,NaN,Seattle Seahawks,...,69,3,18,322,171,151,NaN,12.89,-7.62,-15.43
9,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bye Week,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
